## Assignment 1
### Group 14 

> Github repository: https://github.com/LivDreyer/CSS24.git

> Shortlog of git commits:
- 7  LivDreyer
- 3  AIAndreas
- 2  FelixxAI

The git-commit is a little skewed as some files were sent privately and not commited through the github. Addtionally, some exercises used to generate .csv-files were not included in this assignment meaning they were also not commited.

> Contribution: The workload was distributed equally between all members of the group. 

> Data: The data has been attached in a zip-file with corresponding names for the data loading in this file.

In [73]:
#Importations 
import requests 
from bs4 import BeautifulSoup as BS
import pandas as pd 
import numpy as np
import ast
import networkx as nx
import matplotlib
import requests
from datetime import datetime
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from collections import Counter
import json
from statistics import mode

## Part 1: Web-scrabing 

In this part, we have included keynote speakers, chairs, authors of parallel talks, authors of posters, the names of researchers from the programme committee of the conference, and the organizers of tutorials. 

#### Names from the program: Plenary Speakers, Keynote Speakers, Authors of Parallel Talks, Authors of Posters, and Chairs
#### Link: https://ic2s2-2023.org/program

In [74]:
#Get html to py

link = "https://ic2s2-2023.org/program"
r = requests.get(link)
soup = BS(r.content)

In [75]:
# Plenary, parallel talks, posters speaker
# Name of final list: ppp_names

#find programs from each day 
plen = soup.find_all("ul", {"class":"nav_list"})

#find all names from the day 
names_plen = []
names_plen_text = []
for i in range(len(plen)): 
    names_plen.append(plen[i].find_all("i"))

for i in range(len(names_plen)):
    for j in range(len(names_plen[i])):
        names_plen_text.append(names_plen[i][j].text)

plen_names_lst = [name for names in names_plen_text for name in names.split(', ')]

ppp_names = []
for i in range(len(plen_names_lst)): 
    ppp_names.append(plen_names_lst[i].title())


for name in ppp_names:
    if '\u201a' in name:
        ppp_names.remove(name)

#print(ppp_names)

In [76]:
# Keynote speakers 
# Name of final list: key_names

table = soup.find_all("table", {"class": "tutorials"})
table = table[0].find_all("td")

table_text = []
for i in range(len(table)):
    txt = table[i].text
    if "Keynote" in txt: 
        table_text.append(txt)

key_names = []
for i in table_text: 
    key_names.append(i[10:])


#print(key_names)

In [77]:
# Chairs
# Name of final list: chair_names


chair = soup.find_all("div", {"class":"col-9 col-12-medium"})

#find chairs
names_chairs = []
names_chairs_text = []
for i in range(len(chair)): 
    names_chairs.append(chair[i].find_all("h2"))

for i in range(len(names_chairs)):
    for j in range(len(names_chairs[i])):
        names_chairs_text.append(names_chairs[i][j].text)


chr = []

for i in names_chairs_text:
    if "Chair" in i: 
        chr.append(i)

chair_names = []
for i in chr: 
    idx = i.find("Chair:")
    chair_names.append(i[idx + len("Chair:"):].strip())
    
    
#print(chair_names)

#### Names from the programme committee 
#### Link: https://ic2s2-2023.org/program_committee

In [78]:
link_pc = "https://ic2s2-2023.org/program_committee"
r_pc = requests.get(link_pc)
soup_pc = BS(r_pc.content)

In [79]:
# Programme Committee 
# Name of final list: pcom_names 

pc = soup_pc.find_all("article", {"class":"box post"})

#find all names from the day 
pc_names = []
pc_text = []
for i in range(len(pc)): 
    pc_names.append(pc[i].find_all("li"))

for i in range(len(pc_names)):
    for j in range(len(pc_names[i])):
        pc_text.append(pc_names[i][j].text)


pcom_names = []
for i in range(len(pc_text)):
    lst = pc_text[i].split(" (")
    pcom_names.append(lst[0])

#print(pcom_names)

#### Names from the Tutorials
#### Link: https://ic2s2-2023.org/tutorials

In [80]:
link_t = "https://ic2s2-2023.org/tutorials"
r_t = requests.get(link_t)
soup_t = BS(r_t.content)

In [81]:
# Tutorials 
# Name of final list: tut_names_f

tut = soup_t.find_all("div", {"class": "col-5 col-12-medium"})

#find all names from the day 
tut_names = []
tut_text = []
for i in range(len(tut)): 
    tut_names.append(tut[i].find_all("li"))

for i in range(len(tut_names)):
    for j in range(len(tut_names[i])):
        tut_text.append(tut_names[i][j].text)

#print(tut_text)

tut_names_f = []
for i in range(len(tut_text)):
    lst = tut_text[i].split(",")
    tut_names_f.append(lst[0])

#print(tut_names_f)

### Collect all names in one final string and process 

In [82]:
# Collect all names in one list of strings 

all_names = chair_names + key_names + ppp_names + pcom_names + tut_names_f
#print(all_names)

#Capitalize every name 
all_names_final = []
for i in range(len(all_names)): 
    all_names_final.append(all_names[i].title())

all_names_final = list(set(all_names_final))


#Pandas DF 
names_DF = pd.DataFrame(all_names_final, columns=['Researcher Name'])
names_DF = names_DF.sort_values(by='Researcher Name')

#Split names into first name and last name, dropping middle names in all formats: "John Name Doe" or "John N. Doe"
names_DF['first_name'] = names_DF['Researcher Name'].str.split().str[0]
names_DF['last_name'] = names_DF['Researcher Name'].str.split().str[-1]

#Remove duplicates of names such as "John Name Doe" or "John N. Doe"
names_DF.drop_duplicates(subset=['first_name', 'last_name'], inplace=True)

#Save first column (names) to file
names_DF['Researcher Name'].to_csv('researchers_names.csv', index=False)

#Find number of researchers

print("Number of names including optional exercises: ",len(names_DF))

Number of names including optional exercises:  1620


**How many unique researchers did you get?**

As seen above, we got 1620 unique names of researchers when including the optional exercises. 

**Explain the process you followed to web-scrape the page. Which choices did you make to accurately retreive as many names as possible? Which strategies did you use to assess the quality of your final list? Explain your reasoning and your choices (answer in max 150 words).**

When initially inspecting the webpage to understand how it was structured, and where we should look, we relied on visual cues signaling what html-code corresponded to the data we wanted. Working with retrieving names of researchers from the program, we structured the retrieval in three parts, as we recognized different patterns for each type of name. We repeated this process for the optional exercise. For each part we used the [BeautifulSoup Python package](https://pypi.org/project/beautifulsoup4/)'s .[find_all](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#find-all) method to extract the wanted names, and lastly combined every list of names to one final list. For the final list, we capitalized each name, and then used the set() method to remove apparent duplicates. We then created a Pandas Dataframe, where we split every name into first and last name excluding the middle name. It was then possible to remove rows with identical first and last name, letting us create a csv.file with unique “Researcher Names”.

## Part 2: Ready Made vs Custom Made Data

*Exercise: Ready made data vs Custom made data In this exercise, I want to make sure you have understood they key points of my lecture and the reading.*

**What are pros and cons of the custom-made data used in Centola's experiment (the first study presented in the lecture) and the ready-made data used in Nicolaides's study (the second study presented in the lecture)? You can support your arguments based on the content of the lecture and the information you read in Chapter 2.3 of the book (answer in max 150 words).**

Centola’s experiment [1] allows the investigation of diffusion processes with no influence from confounding variables such as advertising or social reinforcement from outside of the experiment. Though generally desired it’s hard to apply the results directly on real-word settings. Despite a smaller sample size, Centola could repeat the experiment to reproduce tendencies of spread of information. A problem of volunteer bias could however possibly arise from how he recruited participants.
In Nicolaides’s study [2] the large sample size and the non-reactivity of users constitute an advantage when investigating contagion as it can mirror real-world conditions. However, as mentioned in “Bit by bit” 2.3.3 [3], non-reactivity does not ensure that the data directly reflects behavior. Users of the fitness-app is not forced to participate even though they have connections on the app, and to paraphrase from the book: “It’s not that I am not exercising, I am just not posting it on thefitness- app”. 


**How do you think these differences can influence the interpretation of the results in each study? (answer in max 150 words)**

Centola’s findings are based on an artificial environment that find significant results mapping the “essence” of behavioral contagion, however, the reactivity of participants could influence the interpretation of the results. Additionally, as mentioned, the participants were recruited from health-sites which could promote the behavior Centola finds considering his experiment is also concerned with health behavior. 

Depending on how Nicolaides performs the linear regression to examine how one user’s exercise might influence their connections’ exercise on the fitness-tracker app, it might not mirror the actual contagion happening. If people are active users who follow their friends, but do not post their exercise themselves, the results of the linear regression may reflect this more than the actual influence users have on their connections.


[1] Damon Centola ,The Spread of Behavior in an Online Social Network Experiment.Science329,1194-1197(2010).DOI:10.1126/science.1185231

[2] Aral S, Nicolaides C. Exercise contagion in a global social network. Nat Commun. 2017 Apr 18;8:14753. doi: 10.1038/ncomms14753.

[3] Salganik, Matthew J. 2017. Bit by Bit: Social Research in the Digital Age. Princeton, NJ: Princeton University Press. Open review edition.

## Part 3: Gathering Research Articles using the OpenAlex API

### Step 1: Retrieve data. 

**Starting with the authors you identified in Week 2, Exercise 2, use the OpenAlex API works endpoint to fetch the research articles they have authored. For each article, retrieve the following details:**

- _id_: The unique OpenAlex ID for the work.
- _publication_year_: The year the work was published.
- _cited_by_count_: The number of times the work has been cited by other works.
- _author_ids_: The OpenAlex IDs for the authors of the work.
- _title_: The title of the work.
- _abstract_inverted_index_: The abstract of the work, formatted as an inverted index.

In [83]:
# Constants
SOCIAL_SCIENCE_CONCEPTS = {'https://openalex.org/C144024400', 'https://openalex.org/C15744967', 'https://openalex.org/C162324750', 'https://openalex.org/C17744445'}
QUANTITATIVE_CONCEPTS = {'https://openalex.org/C33923547', 'https://openalex.org/C121332964', 'https://openalex.org/C41008148'}

# Session for connection pooling and retries
session = requests.Session()
retries = Retry(total=5, backoff_factor=1, status_forcelist=[429])
session.mount('https://', HTTPAdapter(max_retries=retries))

def work_meets_criteria(work):
    social_science_present = any(concept['id'] in SOCIAL_SCIENCE_CONCEPTS for concept in work.get('concepts', []))
    quantitative_present = any(concept['id'] in QUANTITATIVE_CONCEPTS for concept in work.get('concepts', []))
    return social_science_present and quantitative_present

def fetch_work_for_author(author_id):
    works_details = []
    abstracts_details = []
    cursor = '*'
    author_id_query = f"author.id:{author_id}"
    citations_filter = "cited_by_count:>10"

    while cursor:
        url = f"https://api.openalex.org/works?filter={author_id_query},{citations_filter}&per-page=200&cursor={cursor}"
        response = session.get(url, timeout=10)
        data = response.json()
        works = data.get('results', [])
        for work in works:
            if work_meets_criteria(work):
                author_ids = [authorship['author']['id'] for authorship in work.get('authorships', []) if 'author' in authorship and 'id' in authorship['author']]
                if len(author_ids) < 10:
                    works_details.append({
                        'id': work['id'],
                        'publication_year': work.get('publication_year'),
                        'cited_by_count': work.get('cited_by_count'),
                        'title': work.get('title'),
                        'author_ids': ';'.join(author_ids)
                    })
                    abstracts_details.append({
                        'id': work['id'],
                        'title': work.get('title'),
                        'abstract_inverted_index': work.get('abstract_inverted_index')
                    })
        cursor = data.get('meta', {}).get('next_cursor')
    return works_details, abstracts_details

def fetch_works_and_abstracts_for_authors(author_ids):
    works_details = []
    abstracts_details = []
    with ThreadPoolExecutor(max_workers=10) as executor: 
        futures = [executor.submit(fetch_work_for_author, author_id) for author_id in author_ids]
        for future in as_completed(futures):
            author_works, author_abstracts = future.result()
            works_details.extend(author_works)
            abstracts_details.extend(author_abstracts)
    return works_details, abstracts_details

def parallel_fetch_works_and_abstracts(author_ids, batch_size=20):  
    papers_data = []
    abstracts_data = []
    with ThreadPoolExecutor(max_workers=5) as executor:  
        for i in range(0, len(author_ids), batch_size):
            author_ids_batch = author_ids[i:i+batch_size]
            futures = [executor.submit(fetch_works_and_abstracts_for_authors, author_ids_batch)]
            for future in as_completed(futures):
                works_details, abstracts_details = future.result()
                papers_data.extend(works_details)
                abstracts_data.extend(abstracts_details)
    return papers_data, abstracts_data

**Data Storage:** Organize the retrieved information into two Pandas DataFrames and save them to two files in a suitable format:
- The *IC2S2 papers* dataset should include: *id, publication\_year, cited\_by\_count, author\_ids*.
- The *IC2S2 abstracts* dataset should include: *id, title, abstract\_inverted\_index*.

In [84]:
# Load your author IDs here
researchers_df = pd.read_csv("IC2S2_authors_dataset.csv")
author_ids_filtered = researchers_df[(researchers_df['works_count'] >= 5) & (researchers_df['works_count'] <= 5000)]['id'].apply(lambda x: x.split('/')[-1]).tolist()

# Fetch works and abstracts
papers_data, abstracts_data = parallel_fetch_works_and_abstracts(author_ids_filtered)

# Convert to DataFrame and save
papers_df = pd.DataFrame(papers_data)
abstracts_df = pd.DataFrame(abstracts_data)
papers_df.to_csv('IC2S2_papers_dataset.csv', index=False)
abstracts_df.to_csv('IC2S2_abstracts_dataset.csv', index=False)

**Data Overview and Reflection questions:** Answer the following questions:

**1. Dataset summary.** How many works are listed in your IC2S2 papers dataframe? How many unique researchers have co-authored these works?

Number of works listed in IC2S2 papers dataframe: 10509
Number of unique researchers who have co-authored these works: 14095

Please see the code below as well: 

In [85]:
# Dataset summary
num_works = len(papers_df)
num_unique_researchers = len(set(';'.join(papers_df['author_ids']).split(';')))
print(f"Number of works listed in IC2S2 papers dataframe: {num_works}")
print(f"Number of unique researchers who have co-authored these works: {num_unique_researchers}")

Number of works listed in IC2S2 papers dataframe: 10527
Number of unique researchers who have co-authored these works: 14115


**2. Efficiency in code.** Describe the strategies you implemented to make your code more efficient. How did your approach affect your code's execution time? (answer in max 150 words)

To enhance the efficiency of the code we implemeted a retry mechanism as we kept running into error code 429 which we later learned was too many requests which was a byproduct of the ThreadPoolExecutor. Beacuse of this we ended up also implementing a backoff period which was done to make the requests stop for a moment so we didnt keep running into the 10 request per second hard cap. We also implemented batch requests to streamline the process which also reduced the occurence of error code 429.

**3. Filtering Criteria and Dataset Relevance** 
Reflect on the rationale behind setting specific thresholds for the total number of works by an author, the citation count, the number of authors per work, and the relevance of works to specific fields. How do these filtering criteria contribute to the relevance of the dataset you compiled? Do you believe any aspects of Computational Social Science research might be underrepresented or overrepresented as a result of these choices? (answer in max 150 words)

Setting thresholds for total works, citation count, and number of authors per work ensures that only the most relevant works and authors are included. Take for example the citation count, in the acedemic community citation count is almost directly correlated with relevance and is a sort of unofficial quality-checker. Including works with less than 10 citations risks bringing in works of questionable quality.
The filtering of papers not tagged with one of the relevant fields of study is kind of self explanatory as we do not want non-relevant fields of study in the paper pool. These filters make sure that we do not see an over-representation of non-relevant papers which in the end should be beneficial.

## Part 4: The Network of Computational Social Scientists

### Part 1: Network Construction

**Weighted Edgelist Creation:** Start with your dataframe of papers. Construct a weighted edgelist where each list element is a tuple containing three elements: the author ids of two collaborating authors and the total number of papers they've co-authored. Ensure each author pair is listed only once.

In [86]:
#Load dataframes 


papers_df = pd.read_csv('final_papers_dataset.csv', usecols = ["id","author_ids","cited_by_count","publication_year"])
authors_df = pd.read_csv('final_authors_dataset.csv', usecols = ["id","display_name", "country_code"])

papers_df['author_ids'] = papers_df['author_ids'].apply(lambda x: [url if url.startswith('https://openalex.org/') else 'https://openalex.org/' + url for url in ast.literal_eval(x)] if x.startswith('[') else ['https://openalex.org/' + url if not url.startswith('https://openalex.org/') else url for url in x.split(';')])

In [87]:
#Create List of Collaborations between authors

collabs = []
unique_authors = set()
for work_idx in range(len(papers_df["id"])):
    work = papers_df["id"][work_idx]
    collabs.extend([(a, b) for idx, a in enumerate(papers_df["author_ids"][work_idx]) for b in papers_df["author_ids"][work_idx][idx + 1:]])
    unique_authors.update(papers_df["author_ids"][work_idx])
    if work_idx % 10000 == 0:
        print(work_idx) #To keep track of where we are in the process


#Count the number of occurences of the Collaborations
counter = Counter(collabs)
occurrences = counter.most_common()
authors_collabs = [(pair[0], pair[1], count) for pair, count in occurrences]

0
10000
20000
30000
40000
50000
60000


**Graph Construction:**

- Use NetworkX to create an undirected Graph.
- Employ the add_weighted_edges_from function to populate the graph with the weighted edgelist from step 1, creating a weighted, undirected graph.

**Node Attributes:**

- For each node, add attributes for the author's display name, country, citation count, and the year of their first publication in Computational Social Science. The display name and country can be retrieved from your authors dataset. The year of their first publication and the citation count can be retrieved from the papers dataset.
- Save the network as a JSON file.

In [88]:
#Create dataframe for graph analysis
authors = authors_df[authors_df['id'].isin(unique_authors)]
authors_citations = []
idx = 0

for author_id in authors['id']:

    author_papers = papers_df[papers_df['author_ids'].apply(lambda x: author_id in x)]
    total_citations = author_papers['cited_by_count'].sum()
    first_publication_year = author_papers['publication_year'].min()
    authors_citations.append((author_id, total_citations, first_publication_year))
    idx += 1
    if idx % 1000 == 0: #To keep track of where we are in the process
        print(idx)

authors_citations_df = pd.DataFrame(authors_citations, columns=['Author ID', 'Total Citations', 'First Publication Year'])
combined_df = pd.merge(authors, authors_citations_df, left_on='id', right_on='Author ID')
combined_df = combined_df[['Author ID', 'display_name', 'country_code', 'Total Citations', 'First Publication Year']]
combined_df.columns = ['Author ID', 'display_name', 'country', 'citation_count', 'first_publication']

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [89]:
#Add attributes for each author to list 

tupleattr_list = []
for i in range(len(combined_df)):
    id = combined_df['Author ID'][i]
    author_attribute = {'display_name': combined_df['display_name'][i], 'country': combined_df['country'][i], 'citation_count': int(combined_df['citation_count'][i]), 'first_publication': int(combined_df['first_publication'][i])}
    tupleattr_list.append((id, author_attribute))


In [90]:
#Create graph and employ the G.add_weighted_edges_from() function
G = nx.Graph()
G.add_nodes_from(tupleattr_list)

#Check that we have the correct other before creating edges
authors_ids = set(authors['id'])
authors_collabs = [(a, b, count) for a, b, count in authors_collabs if a in authors_ids and b in authors_ids]

G.add_weighted_edges_from(authors_collabs)

In [91]:
#Save the network as a JSON file
data = nx.node_link_data(G)

with open('graph.json', 'w') as f:
    json.dump(data, f, indent=4)

### Part 2: Preliminary Network Analysis 

**Network Metrics**

**1. What is the total number of nodes (authors) and links (collaborations) in the network?**

Utilizing [G.number_of_nodes()](https://networkx.org/documentation/stable/tutorial.html) and [G.number_of_edges()](https://networkx.org/documentation/stable/tutorial.html) where G is our network graph, we find a total of 14100 nodes (authors) and 49431 links (collaborations) in the network. 

**2. Calculate the network's density (the ratio of actual links to the maximum possible number of links). Would you say that the network is sparse? Justify your answer.**

When calculating the maximum number of links possible in an undirected graph using the formula L_max=(n*(n-1))/2, where n denotes number of nodes, we find that the maximum number of links for this graph is 99397950. As mentioned above, our graph has a total of 49431 links equal to our network having a density of 0.000497. We would say that the network is sparse as the number of actual links is much smaller than that of L_max, which is to be expected from real-life networks such as this one [4]. 

[4] Network Science. Albert-László Barabási. 2016. Cambridge United Kingdom: Cambridge University Press.

**3. Is the network fully connected (i.e., is there a direct or indirect path between every pair of nodes within the network), or is it disconnected?**

Using the length of [list(nx.connected_components(G))](https://networkx.org/documentation/stable/tutorial.html) we see that the graph can be partitioned into multiple connected components meaning the network is considered disconnected. 

**4. If the network is disconnected, how many connected components does it have? A connected component is defined as a subset of nodes within the network where a path exists between any pair of nodes in that subset.**

Using the above method, and given that the network is disconnected, we find that there are 118 connected components. Please see code below as well:

In [92]:
print("The network has", len(list(nx.connected_components(G))), "connected components")

The network has 118 connected components


**5. How many isolated nodes are there in your network? An isolated node is defined as a node with no connections to any other node in the network.**

Using the code in the cell below, we see that there are no isolated nodes in the graph as there are 11 nodes where [G.degree\[node\]](https://networkx.org/documentation/stable/tutorial.html) is equal to 0. 

In [93]:
isolated = 0
for i in range(len(combined_df['Author ID'])):
    if G.degree(combined_df['Author ID'][i]) == 0:
        isolated +=1
print("Number of isolated nodes:", isolated)

Number of isolated nodes: 11


**6. Discuss the results above on network density, and connectivity. Are your findings in line with what you expected? Why? (answer in max 150 words)**

We see that the density is quite low, which can be expected from a real-life network, and that within the 14100 authors we initially found, we have seen 118 connected components. As this network represents collaborations on papers between authors, we would expect this level of connectivity and density as research, and the work around writing and publishing papers is quite time-heavy. This number also reflects that the authors included work in different fields of research such as Sociology, Psychology, Economics, and Political Science, have different nationalities, and different places of work. 

**Degree Analysis:**

**Compute the average, median, mode, minimum, and maximum degree of the nodes. Perform the same analysis for node strength (weighted degree). What do these metrics tell us about the network? (answer in max 150 words)**

Please see the code below for the computation of each descriptive statistic. 

The network of published work in Computational Social Science and its authors suggests that most authors may collaborate with a few others; this is indicated by the fact that the average weighted degree is higher than the average degree, highlighting significant repeated collaborations among some authors. Conversely, the presence of nodes with a degree of 0 indicates that some authors have not collaborated within the network at all. For both non-weighted and weighted degrees, the large gap between the median and maximum degrees also suggests the existence of a few large and centralized hubs, as well as numerous peripheral ones.

In [94]:
print("Degrees of nodes:")
degrees = [G.degree(n) for n in G.nodes()]
print("The average degree is:", np.mean(degrees))
print("The median degree is:", np.median(degrees))
print("The minimum degree is:", min(degrees))
print("The maximum degree is:", max(degrees))
print("The mode degree is:", mode(degrees))
print(" ")
print("Weighted degrees of nodes:")
weighted_degrees = [G.degree(n, weight='weight') for n in G.nodes()]
print("The average weighted degree is:", np.mean(weighted_degrees))
print("The median weighted degree is:", np.median(weighted_degrees))
print("The minimum weighted degree is:", min(weighted_degrees))
print("The maximum weighted degree is:", max(weighted_degrees))
print("The mode weighted degree is:", mode(weighted_degrees))

Degrees of nodes:
The average degree is: 7.011489361702128
The median degree is: 5.0
The minimum degree is: 0
The maximum degree is: 169
The mode degree is: 4
 
Weighted degrees of nodes:
The average weighted degree is: 9.173191489361702
The median weighted degree is: 6.0
The minimum weighted degree is: 0
The maximum weighted degree is: 264
The mode weighted degree is: 4


**Top Authors:**

**1. Identify the top 5 authors by degree. What role do these nodes play in the network?**

The top 5 authors by degree is: 

- **Qin Li with degree = 169** from Central South University in China 
- **Mariano Sigman with degree = 155** from Universidad Torcuato Di Tella in Argentina
- **Stephan Lewandowsky with degree = 150** from University of Bristol, University of Western Australia, and University of Potsdam
- **Denny Borsboom with degree = 140** from University of Amsterdam
- **Dan Jurafsky with degree = 129** from Stanford University

These are authors who have collaborated with many other authors on papers included in the data we are working with. This could be due to them having special expertise within a certain area detrimental to other researchers, or that they have supervised other researchers in their projects. They could be the nodes who connects clusters of researchers that otherwise would not have been connected, which would show up in the number of connected components. 

In [95]:
degrees = G.degree()
top_5_nodes = sorted(degrees, key=lambda x: x[1], reverse=True)[:5]
top_5_nodes = [(node, degree) for node, degree in top_5_nodes]
print("Nodes with highest degrees:", top_5_nodes)

Nodes with highest degrees: [('https://openalex.org/A5075080019', 169), ('https://openalex.org/A5005493160', 155), ('https://openalex.org/A5017914184', 150), ('https://openalex.org/A5029100305', 140), ('https://openalex.org/A5087088138', 129)]


**2. Research these authors online. What areas do they specialize in? Do you think that their work aligns with the themes of Computational Social Science? If not, what could be possible reasons? (answer in max 150 words)**

[Qin Li](https://scholar.google.com/citations?user=6-u5zzMAAAAJ&hl=en) is a PhD, School of Energy Science and Engineering. [Mariano Sigman](https://www.marianosigman.org/en/) is a neuroscientist and author. [Stephan Lewandowsky](https://en.wikipedia.org/wiki/Stephan_Lewandowsky) is a psychologist who is currently head of Cognitive Psychology at University of Bristol, and whos work was originally on "computer simulations of people's decision-making processes". [Denny Borsboom](https://dennyborsboom.com) is also a psychologist and psychometrician who use network theory for his work. [Dan Jurafsky](https://web.stanford.edu/~jurafsky/) is a professor in linguistics and computer science, who study NLP and its application in social and cognitive sciences. 

As we see, these authors are not all within fields of research that are exclusivly within the themes of Computational Social Science. As mentioned in the previous question, their degrees (within the network) could possibly be explained by their expertise within their respective fields which researchers whos work align with the themes of Computational Social Science could benefit from. 